In [121]:
from pathlib import Path
import numpy as np
from numba import jit
from utils.get_input import get_input
from io import StringIO
import pandas as pd


inp = Path("./test_input").read_text()
inp = get_input(8)

In [122]:
# %%timeit
# Structure input
inp_np = np.array([list(line) for line in inp.split("\n")])
print(inp_np)
print(np.flipud(inp_np))

antennas = np.where(inp_np != '.')
antenna_types = inp_np[antennas]

data = np.array([antennas[0], antennas[1], antenna_types]).T
data = pd.DataFrame(data, columns=["y", "x", "type"])
data["x"] = data["x"].astype(int)
data["y"] = data["y"].astype(int)
data

[['.' 'C' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['u' '.' '.' ... '.' 'a' '.']
 ...
 ['.' '.' '.' ... '.' '.' 'v']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']]
[['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' 'v']
 ...
 ['u' '.' '.' ... '.' 'a' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' 'C' '.' ... '.' '.' '.']]


,y,x,type
0,0,1,C
1,0,17,w
2,0,41,M
3,0,43,E
4,1,15,G
...,...,...,...
226,46,41,v
227,47,49,v
228,49,17,z
229,49,19,W


In [123]:
antinodes

[(3, 2),
 (0, 11),
 (11, 3),
 (9, 4),
 (7, 5),
 (5, 6),
 (7, 0),
 (5, 11),
 (4, 9),
 (1, 3),
 (0, 1),
 (10, 1),
 (8, 2),
 (6, 3),
 (0, 6),
 (5, 1),
 (2, 10),
 (11, 10),
 (2, 4),
 (1, 3),
 (11, 11),
 (10, 10),
 (7, 7),
 (6, 6),
 (5, 5),
 (4, 4),
 (3, 3),
 (2, 2),
 (1, 1),
 (0, 0)]

In [140]:
import itertools


def part1():
    max_y, max_x = inp_np.shape
    antinodes = []
    for _, t in data.groupby("type"):
        ant_in = t.index
        ant_combs = list(itertools.combinations(ant_in, 2))
        for ant1, ant2 in ant_combs:
            t_1 = t.loc[ant1]
            t_2 = t.loc[ant2]
            diff_x = t_1["x"] - t_2["x"]
            dir_x = diff_x / abs(diff_x)
            diff_y = t_1["y"] - t_2["y"]
            dir_y = diff_y / abs(diff_y)
            a_1 = (
                int(t_1["y"] + abs(diff_y) * dir_y),
                int(t_1["x"] + abs(diff_x) * dir_x),
            )
            a_2 = (
                int(t_2["y"] + abs(diff_y) * -1 * dir_y),
                int(t_2["x"] + abs(diff_x) * -1 * dir_x),
            )
            if a_1[0] >= 0 and a_1[0] < max_y and a_1[1] >= 0 and a_1[1] < max_x:
                antinodes.append(a_1)
            if a_2[0] >= 0 and a_2[0] < max_y and a_2[1] >= 0 and a_2[1] < max_x:
                antinodes.append(a_2)

    return len(set(antinodes))


part1()

371

In [ ]:
%%timeit
part1()

In [ ]:
import itertools


def part2():
    max_y, max_x = inp_np.shape

    antinodes = []
    for _, t in data.groupby("type"):
        ant_in = t.index
        ant_combs = list(itertools.combinations(ant_in, 2))
        for ant1, ant2 in ant_combs:
            t_1 = t.loc[ant1]
            t_2 = t.loc[ant2]
            diff_x = t_1["x"] - t_2["x"]
            dir_x = diff_x / abs(diff_x)
            diff_y = t_1["y"] - t_2["y"]
            dir_y = diff_y / abs(diff_y)

            for i in range(-1000, 1000):
                a = (
                    int(t_1["y"] + abs(diff_y) * dir_y * i),
                    int(t_1["x"] + abs(diff_x) * dir_x * i),
                )
                if a[0] >= 0 and a[0] < max_y and a[1] >= 0 and a[1] < max_x:
                    antinodes.append(a)

    return len(set(antinodes))


part2()

1229

In [134]:
%%timeit
part2()

1.5 s ± 4.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [138]:
# Print result
inp_np2 = inp_np.copy()
for x,y in antinodes:
    inp_np2[int(x), int(y)] = "#"

inp_np2

array([['#', '#', '.', ..., '.', '#', '.'],
       ['#', '.', '#', ..., '#', '.', '.'],
       ['#', '.', '.', ..., '.', '#', '#'],
       ...,
       ['#', '.', '.', ..., '.', '.', '#'],
       ['.', '.', '#', ..., '.', '.', '.'],
       ['#', '.', '.', ..., '.', '#', '.']], dtype='<U1')